In [ ]:
import gradio as gr
import os
import sys
import io
import shutil
from ultralytics import YOLO

# Mapping from display name to video folder and model
VIDEO_MODEL_MAP = {
    "Actual_IR": {
        "path": "/home/samy/shivam/dsr_project/salguided_od_datasets/video_demo/Actual_ir",
        "model": "/home/samy/shivam/dsr_project/Project_files/runs/yolov11_ir_d3/train_n/weights/best.pt"
    },
    "Vehicle_IR": {
        "path": "/home/samy/shivam/dsr_project/salguided_od_datasets/video_demo/Vehicle_ir",
        "model": "/home/samy/shivam/dsr_project/Project_files/runs/yolov11_ir_d4/train_n/weights/best.pt"
    }
}

# List video files in a folder
def list_videos(folder):
    return [f for f in os.listdir(folder) if f.lower().endswith(('.mp4', '.avi', '.mov'))]

# Tracking inference with log capture and robust output handling
import subprocess
import uuid

def run_tracking(folder_key, video_name, tracker_type):
    if not folder_key or not video_name:
        return None, "Error: Please select both a video folder and a video file."

    info = VIDEO_MODEL_MAP[folder_key]
    video_path = os.path.join(info['path'], video_name)
    model = YOLO(info['model'])

    buf = io.StringIO()
    sys_stdout = sys.stdout
    sys_stderr = sys.stderr
    sys.stdout = sys.stderr = buf

    try:
        tracker_cfg = tracker_type if tracker_type.endswith('.yaml') else f"{tracker_type}.yaml"
        results = model.track(source=video_path, tracker=tracker_cfg, persist=True, save=True)
    finally:
        sys.stdout = sys_stdout
        sys.stderr = sys_stderr

    log_output = buf.getvalue()
    res = results[0]
    save_dir = getattr(res, 'save_dir', None)
    if not save_dir:
        return None, "Could not determine save directory."

    # Find the output video
    tracked_path = None
    for f in os.listdir(save_dir):
        if f.lower().endswith((".mp4", ".avi", ".mov")):
            tracked_path = os.path.join(save_dir, f)
            break

    if not tracked_path or not os.path.exists(tracked_path):
        return None, f"Error: Output video not found in {save_dir}. Logs:\n" + log_output

    # Convert to Gradio-compatible .mp4 using ffmpeg
    output_final = os.path.join(os.getcwd(), f"tracked_{uuid.uuid4().hex[:6]}.mp4")
    try:
        subprocess.run([
            "ffmpeg", "-y", "-i", tracked_path,
            "-vcodec", "libx264", "-pix_fmt", "yuv420p", "-an",
            output_final
        ], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    except Exception as e:
        return None, f"ffmpeg conversion failed: {e}\n\nLogs:\n{log_output}"

    return output_final, log_output

# Build Gradio interface for video demo
def create_video_demo():
    with gr.Blocks() as demo:
        gr.Markdown("## Surveillance Demo - Video Tracking Mode")

        with gr.Row():
            folder_dd = gr.Dropdown(list(VIDEO_MODEL_MAP.keys()), label="Select Video Folder")
            video_dd = gr.Dropdown(choices=[], label="Select Video File", allow_custom_value=False)

        tracker_radio = gr.Radio(choices=["bytetrack", "botsort"], value="bytetrack", label="Tracker Type")
        run_btn = gr.Button("Run Tracking")

        output_video = gr.Video(label="Tracked Output Video")
        log_area = gr.Textbox(label="Tracking Logs", lines=20, interactive=False)

        # Update video dropdown when folder changes
        def update_videos(folder_key):
            if folder_key in VIDEO_MODEL_MAP:
                vids = list_videos(VIDEO_MODEL_MAP[folder_key]['path'])
                return gr.update(choices=vids, value=vids[0] if vids else None)
            return gr.update(choices=[], value=None)

        folder_dd.change(fn=update_videos, inputs=folder_dd, outputs=video_dd)
        run_btn.click(fn=run_tracking, inputs=[folder_dd, video_dd, tracker_radio], outputs=[output_video, log_area])

    return demo

if __name__ == '__main__':
    demo = create_video_demo()
    demo.launch(share=True)


In [ ]:
import gradio as gr
import os
import sys
import io
import shutil
from ultralytics import YOLO

# Mapping from display name to video folder and model
VIDEO_MODEL_MAP = {
    "Actual_IR": {
        "path": "/home/samy/shivam/dsr_project/salguided_od_datasets/video_demo/Actual_ir",
        "model": "/home/samy/shivam/dsr_project/Project_files/runs/yolov11_ir_d3/train_n/weights/best.pt"
    },
    "Vehicle_IR": {
        "path": "/home/samy/shivam/dsr_project/salguided_od_datasets/video_demo/Vehicle_ir",
        "model": "/home/samy/shivam/dsr_project/Project_files/runs/yolov11_ir_d4/train_n/weights/best.pt"
    }
}

# List video files in a folder
def list_videos(folder):
    return [f for f in os.listdir(folder) if f.lower().endswith(('.mp4', '.avi', '.mov'))]

# Tracking inference with log capture and robust output handling
import subprocess
import uuid

def run_tracking(folder_key, video_name, tracker_type):
    if not folder_key or not video_name:
        return None, "Error: Please select both a video folder and a video file."

    info = VIDEO_MODEL_MAP[folder_key]
    video_path = os.path.join(info['path'], video_name)
    model = YOLO(info['model'])


    # Capture stdout, stderr, and logging
    import logging
    import contextlib

    log_stream = io.StringIO()

    @contextlib.contextmanager
    def capture_logs():
        logger = logging.getLogger()
        log_handler = logging.StreamHandler(log_stream)
        logger.addHandler(log_handler)
        try:
            yield
        finally:
            logger.removeHandler(log_handler)

    tracker_cfg = tracker_type if tracker_type.endswith('.yaml') else f"{tracker_type}.yaml"
    with capture_logs():
        results = model.track(source=video_path, tracker=tracker_cfg, persist=True, save=True)

    log_output = log_stream.getvalue()

    







    res = results[0]
    save_dir = getattr(res, 'save_dir', None)
    if not save_dir:
        return None, "Could not determine save directory."

    # Find the output video
    tracked_path = None
    for f in os.listdir(save_dir):
        if f.lower().endswith((".mp4", ".avi", ".mov")):
            tracked_path = os.path.join(save_dir, f)
            break

    if not tracked_path or not os.path.exists(tracked_path):
        return None, f"Error: Output video not found in {save_dir}. Logs:\n" + log_output

    # Convert to Gradio-compatible .mp4 using ffmpeg
    output_final = os.path.join(os.getcwd(), f"tracked_{uuid.uuid4().hex[:6]}.mp4")
    try:
        subprocess.run([
            "ffmpeg", "-y", "-i", tracked_path,
            "-vcodec", "libx264", "-pix_fmt", "yuv420p", "-an",
            output_final
        ], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    except Exception as e:
        return None, f"ffmpeg conversion failed: {e}\n\nLogs:\n{log_output}"

    return output_final, log_output

# Build Gradio interface for video demo
def create_video_demo():
    with gr.Blocks() as demo:
        gr.Markdown("## Surveillance Demo - Video Tracking Mode")

        with gr.Row():
            folder_dd = gr.Dropdown(list(VIDEO_MODEL_MAP.keys()), label="Select Video Folder")
            video_dd = gr.Dropdown(choices=[], label="Select Video File", allow_custom_value=False)

        tracker_radio = gr.Radio(choices=["bytetrack", "botsort"], value="bytetrack", label="Tracker Type")
        run_btn = gr.Button("Run Tracking")

        output_video = gr.Video(label="Tracked Output Video")
        log_area = gr.Textbox(label="Tracking Logs", lines=20, interactive=False)

        # Update video dropdown when folder changes
        def update_videos(folder_key):
            if folder_key in VIDEO_MODEL_MAP:
                vids = list_videos(VIDEO_MODEL_MAP[folder_key]['path'])
                return gr.update(choices=vids, value=vids[0] if vids else None)
            return gr.update(choices=[], value=None)

        folder_dd.change(fn=update_videos, inputs=folder_dd, outputs=video_dd)
        run_btn.click(fn=run_tracking, inputs=[folder_dd, video_dd, tracker_radio], outputs=[output_video, log_area])

    return demo

if __name__ == '__main__':
    demo = create_video_demo()
    demo.launch(share=True)


In [ ]:
import gradio as gr
import os
import sys
import io
import shutil
from ultralytics import YOLO

# Mapping from display name to video folder and model
VIDEO_MODEL_MAP = {
    "Actual_IR": {
        "path": "/home/samy/shivam/dsr_project/salguided_od_datasets/video_demo/Actual_ir",
        "model": "/home/samy/shivam/dsr_project/Project_files/runs/yolov11_ir_d3/train_n/weights/best.pt"
    },
    "Vehicle_IR": {
        "path": "/home/samy/shivam/dsr_project/salguided_od_datasets/video_demo/Vehicle_ir",
        "model": "/home/samy/shivam/dsr_project/Project_files/runs/yolov11_ir_d4/train_n/weights/best.pt"
    }
}

# List video files in a folder
def list_videos(folder):
    return [f for f in os.listdir(folder) if f.lower().endswith(('.mp4', '.avi', '.mov'))]

# Tracking inference with log capture and robust output handling
import subprocess
import uuid

from collections import defaultdict
import cv2
import numpy as np
import uuid
import subprocess

def run_tracking(folder_key, video_name, tracker_type):
    if not folder_key or not video_name:
        return None, "Error: Please select both a video folder and a video file."

    info = VIDEO_MODEL_MAP[folder_key]
    video_path = os.path.join(info['path'], video_name)
    model = YOLO(info['model'])

    # Prepare tracker
    tracker_cfg = tracker_type if tracker_type.endswith('.yaml') else f"{tracker_type}.yaml"

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return None, f"Failed to open video: {video_path}"

    track_history = defaultdict(list)

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out_path = f"tracked_{uuid.uuid4().hex[:6]}.mp4"
    out = cv2.VideoWriter(out_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    while True:
        success, frame = cap.read()
        if not success:
            break

        results = model.track(frame, tracker=tracker_cfg, persist=True)[0]

        if results.boxes and results.boxes.is_track:
            boxes = results.boxes.xywh.cpu()
            track_ids = results.boxes.id.int().cpu().tolist()

            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))
                if len(track) > 30:
                    track.pop(0)

                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(frame, [points], isClosed=False, color=(0, 255, 0), thickness=2)

        frame = results.plot()
        out.write(frame)

    cap.release()
    out.release()

    # Convert for Gradio
    final_output = os.path.join(os.getcwd(), f"gradio_{uuid.uuid4().hex[:6]}.mp4")
    try:
        subprocess.run([
            "ffmpeg", "-y", "-i", out_path,
            "-vcodec", "libx264", "-pix_fmt", "yuv420p", "-an",
            final_output
        ], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    except Exception as e:
        return None, f"ffmpeg conversion failed: {e}"

    return final_output, "Trajectory tracking completed. No logs — visual feedback only."

# Build Gradio interface for video demo
def create_video_demo():
    with gr.Blocks() as demo:
        gr.Markdown("## Surveillance Demo - Video Tracking Mode")

        with gr.Row():
            folder_dd = gr.Dropdown(list(VIDEO_MODEL_MAP.keys()), label="Select Video Folder")
            video_dd = gr.Dropdown(choices=[], label="Select Video File", allow_custom_value=False)

        tracker_radio = gr.Radio(choices=["bytetrack", "botsort"], value="bytetrack", label="Tracker Type")
        run_btn = gr.Button("Run Tracking")

        output_video = gr.Video(label="Tracked Output Video")
        log_area = gr.Textbox(label="Tracking Logs", lines=20, interactive=False)

        # Update video dropdown when folder changes
        def update_videos(folder_key):
            if folder_key in VIDEO_MODEL_MAP:
                vids = list_videos(VIDEO_MODEL_MAP[folder_key]['path'])
                return gr.update(choices=vids, value=vids[0] if vids else None)
            return gr.update(choices=[], value=None)

        folder_dd.change(fn=update_videos, inputs=folder_dd, outputs=video_dd)
        run_btn.click(fn=run_tracking, inputs=[folder_dd, video_dd, tracker_radio], outputs=[output_video, log_area])

    return demo

if __name__ == '__main__':
    demo = create_video_demo()
    demo.launch(share=True)


In [5]:
import gradio as gr
import os
import sys
import io
import shutil
from ultralytics import YOLO

# Mapping from display name to video folder and model
VIDEO_MODEL_MAP = {
    "Actual_IR": {
        "path": "/home/samy/shivam/dsr_project/salguided_od_datasets/video_demo/Actual_ir",
        "model": "/home/samy/shivam/dsr_project/Project_files/runs/yolov11_ir_d3/train_n/weights/best.pt"
    },
    "Vehicle_IR": {
        "path": "/home/samy/shivam/dsr_project/salguided_od_datasets/video_demo/Vehicle_ir",
        "model": "/home/samy/shivam/dsr_project/Project_files/runs/yolov11_ir_d4/train_n/weights/best.pt"
    }
}

# List video files in a folder
def list_videos(folder):
    return [f for f in os.listdir(folder) if f.lower().endswith(('.mp4', '.avi', '.mov'))]

# Tracking inference with log capture and robust output handling
import subprocess
import uuid

from collections import defaultdict
import cv2
import numpy as np
import uuid
import subprocess

def run_tracking(folder_key, video_name, tracker_type, show_trajectories):
    if not folder_key or not video_name:
        return None, None, "Error: Please select both a video folder and a video file."

    info = VIDEO_MODEL_MAP[folder_key]
    video_path = os.path.join(info['path'], video_name)
    model = YOLO(info['model'])

    tracker_cfg = tracker_type if tracker_type.endswith('.yaml') else f"{tracker_type}.yaml"

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return None, None, f"Failed to open video: {video_path}"

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    out_path = f"tracked_{uuid.uuid4().hex[:6]}.mp4"
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(out_path, fourcc, fps, (width, height))

    track_history = defaultdict(list)

    while True:
        success, frame = cap.read()
        if not success:
            break

        results = model.track(frame, tracker=tracker_cfg, persist=True)[0]

        if results.boxes and results.boxes.is_track:
            boxes = results.boxes.xywh.cpu()
            track_ids = results.boxes.id.int().cpu().tolist()

            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))
                if len(track) > 60:
                    track.pop(0)

                if show_trajectories:
                    points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                    cv2.polylines(frame, [points], isClosed=False, color=(0, 255, 0), thickness=2)

        frame = results.plot()
        out.write(frame)

    cap.release()
    out.release()

    # Convert video for Gradio
    final_output = os.path.join(os.getcwd(), f"gradio_{uuid.uuid4().hex[:6]}.mp4")
    try:
        subprocess.run([
            "ffmpeg", "-y", "-i", out_path,
            "-vcodec", "libx264", "-pix_fmt", "yuv420p", "-an",
            final_output
        ], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    except Exception as e:
        return None, None, f"ffmpeg conversion failed: {e}"

    # Create trajectory map image
    traj_canvas = 255 * np.ones((height, width, 3), dtype=np.uint8)  # white background
    for track in track_history.values():
        if len(track) > 1:
            points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))
            cv2.polylines(traj_canvas, [points], isClosed=False, color=(0, 0, 255), thickness=2)

    return final_output, traj_canvas[:, :, ::-1], "Tracking completed with trajectories."  # BGR to RGB


# Build Gradio interface for video demo
def create_video_demo():
    with gr.Blocks() as demo:
        gr.Markdown("## Surveillance Demo - Video Tracking Mode")

        with gr.Row():
            folder_dd = gr.Dropdown(list(VIDEO_MODEL_MAP.keys()), label="Select Video Folder")
            video_dd = gr.Dropdown(choices=[], label="Select Video File", allow_custom_value=False)

        with gr.Row():
            tracker_radio = gr.Radio(choices=["bytetrack", "botsort"], value="bytetrack", label="Tracker Type")
            show_traj = gr.Checkbox(label="Overlay Trajectories on Video", value=True)

        run_btn = gr.Button("Run Tracking")

        with gr.Row():
            output_video = gr.Video(label="Tracked Output Video")
            traj_image = gr.Image(label="Trajectory Map", type="numpy")

        log_area = gr.Textbox(label="Status", lines=3, interactive=False)

        # Update video list when folder changes
        def update_videos(folder_key):
            if folder_key in VIDEO_MODEL_MAP:
                vids = list_videos(VIDEO_MODEL_MAP[folder_key]['path'])
                return gr.update(choices=vids, value=vids[0] if vids else None)
            return gr.update(choices=[], value=None)

        folder_dd.change(fn=update_videos, inputs=folder_dd, outputs=video_dd)
        run_btn.click(fn=run_tracking,
                      inputs=[folder_dd, video_dd, tracker_radio, show_traj],
                      outputs=[output_video, traj_image, log_area])

    return demo


if __name__ == '__main__':
    demo = create_video_demo()
    demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7865
* Running on public URL: https://b74eac235e1925a74b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



0: 384x640 3 Persons, 21.5ms
Speed: 1.1ms preprocess, 21.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Persons, 2.0ms
Speed: 0.7ms preprocess, 2.0ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Persons, 2.4ms
Speed: 0.8ms preprocess, 2.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Persons, 2.0ms
Speed: 0.7ms preprocess, 2.0ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Persons, 2.0ms
Speed: 0.6ms preprocess, 2.0ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Persons, 2.0ms
Speed: 0.6ms preprocess, 2.0ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Persons, 2.1ms
Speed: 0.6ms preprocess, 2.1ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Persons, 2.2ms
Speed: 0.6ms preprocess, 2.2ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640